In [1]:
import numpy as np
import pandas as pd
import pickle
import networkx as nx
import matplotlib.pyplot as plt

## load string db and get combined score ppi network

In [28]:
############### create mapped graph ppi 
local_stringdb = 'D:/study/thesis/project/HBDM-main/data/nn_data/stringdb/'
# load local STRING database and names
df = pd.read_csv(local_stringdb+'9606.protein.info.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
df['preferred_name'] = df['preferred_name'].str.upper()
stringId2name = df.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = df.set_index('preferred_name')['#string_protein_id'].to_dict()
df = pd.read_csv(local_stringdb+'9606.protein.aliases.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
df['alias'] = df['alias'].str.upper()
aliases2stringId = df.set_index('alias')['#string_protein_id'].to_dict()
def convert_stringId(alias):
    try:
        stringId = name2stringId[alias]
    except:
        #print(alias, 'can\'t be converted by name2stringId! Now trying aliases2stringId.')
        try:
            stringId = aliases2stringId[alias]
        except:
            #print(alias, 'can\'t be converted by aliases2stringId! Now return None.')
            stringId = None
    #print(alias, stringId)
    return stringId

# #string_score_transform = lambda x: -np.log(x/1000)

# graph_df = pd.read_csv(local_stringdb+'9606.protein.physical.links.detailed.v12.0.txt', sep=' ', header=0).convert_dtypes().replace(0, float('nan'))
# #network['combined_score'] = network['combined_score'].apply(string_score_transform)

# graph_df = graph_df[['protein1', 'protein2','combined_score']]
# graph_df['protein1'] = graph_df['protein1'].str.replace('9606.ENSP', '')
# graph_df['protein2'] = graph_df['protein2'].str.replace('9606.ENSP', '')

with open(r'D:\study\thesis\project\HBDM-main\data\datasets\ppi\ppi_index.pkl', 'rb') as f:
    value_to_index_mapping = pickle.load(f)
len(value_to_index_mapping)
# sub_graph_df = graph_df[['protein1', 'protein2']]
# sub_graph_array = np.array(sub_graph_df.to_numpy(), dtype=int)
# mapped_arr = np.vectorize(value_to_index_mapping.get)(sub_graph_array)
# graph_df['node1'] = mapped_arr.T[0]
# graph_df['node2'] = mapped_arr.T[1]
# graph_df.to_csv(r'D:\study\thesis\project\HBDM-main\data\graph_ppi.csv',index=False)

In [24]:
graph_df = pd.read_csv(r'D:\study\thesis\project\HBDM-main\data\graph_ppi.csv')
graph_final = pd.DataFrame()
graph_final['node1'] = graph_df['protein1'].apply(lambda x: f'9606.ENSP{str(x).zfill(11)}')
graph_final['node2'] = graph_df['protein2'].apply(lambda x: f'9606.ENSP{str(x).zfill(11)}')
graph_final['relation'] = 'pp'
graph_final['weight'] = graph_df['combined_score']

In [12]:
st_df = pd.read_csv(r'D:\study\thesis\project\HBDM-main\data\st_graph.csv')
st_df = st_df.dropna()
st_df = st_df[st_df['coexp_value'] != 0]
sub_df = pd.DataFrame()
sub_df['node1'] = st_df['protein1']
sub_df['node2'] = st_df['protein2']
sub_df['relation'] = 'coexp'

In [26]:
result = pd.concat([graph_final, sub_df])

In [29]:
string2name = {value: key for key, value in name2stringId.items()}

In [39]:
nodes = result['node1'].unique().tolist()

In [40]:
nodes.extend(result['node2'].unique().tolist())


In [41]:
node_info = pd.DataFrame()
node_info['node'] = list(set(nodes))

In [43]:
node_info['protein_name'] = node_info['node'].map(string2name)

In [45]:
ppi4d = pd.read_csv(r'D:\study\thesis\project\HBDM-main\data\ppi4D_cluster.csv')

In [47]:
with open(r'D:\study\thesis\project\HBDM-main\data\datasets\ppi\ppi_index.pkl', 'rb') as f:
    value_to_index_mapping = pickle.load(f)
len(value_to_index_mapping)

18767

In [48]:
index2string = {value: key for key, value in value_to_index_mapping.items()}

In [49]:
ppi4d['string'] = ppi4d['node'].map(index2string)

In [50]:
ppi4d['node'] = ppi4d['string'].apply(lambda x: f'9606.ENSP{str(x).zfill(11)}')

In [51]:
node_info = node_info.merge(ppi4d)

In [54]:
node_info.columns

Index(['node', 'protein_name', 'level_1', 'level_2', 'level_3', 'level_4',
       'level_5', 'level_6', 'level_7', 'level_8', 'level_9', 'level_10',
       'level_11', 'level_12', 'string'],
      dtype='object')

In [55]:
node_info = node_info[['node', 'protein_name', 'level_1', 'level_2', 'level_3', 'level_4',
       'level_5', 'level_6', 'level_7', 'level_8', 'level_9', 'level_10',
       'level_11', 'level_12']]

In [65]:
graph_final.columns

Index(['node1', 'node2', 'relation', 'weight'], dtype='object')

In [66]:
graph_final = graph_final[['node1', 'relation', 'node2', 'weight']]

In [84]:
output_file1 = r'D:\study\thesis\project\HBDM-main\data\ppi4d_visual_3000.csv'
output_file2 = r'D:\study\thesis\project\HBDM-main\data\ppi4dnode_visual.csv'
graph_final[:3000].to_csv(output_file1, index=False)
node_info.to_csv(output_file2, index=False)

In [69]:
import networkx as nx

In [71]:
multirelational_graph = nx.MultiGraph()

# Add nodes to the MultiGraph using information from node_df
for index, row in node_info.iterrows():
    node_id = row['node']
    node_attributes = {col: row[col] for col in node_info.columns if col != 'node'}
    # Add the node to the MultiGraph with its attributes
    multirelational_graph.add_node(node_id, **node_attributes)

# Iterate through the network_df and add edges to the MultiGraph
for index, row in graph_final.iterrows():
    start_node = row['node1']
    target_node = row['node2']
    edge_type = row['relation']
    edge_weight = row['weight']

    # Add an edge to the MultiGraph with the specified edge type and weight
    multirelational_graph.add_edge(start_node, target_node, key=edge_type, weight=edge_weight)

Exception ignored in: <function ZipFile.__del__ at 0x00000256B99C9A60>
Traceback (most recent call last):
  File "c:\Users\ZZY\.conda\envs\rescal\lib\zipfile.py", line 1821, in __del__
    self.close()
  File "c:\Users\ZZY\.conda\envs\rescal\lib\zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
